In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
from pyspark.sql import SparkSession

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar

--2022-04-10 22:21:12--  https://jdbc.postgresql.org/download/postgresql-42.2.25.jre7.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984862 (962K) [application/java-archive]
Saving to: ‘postgresql-42.2.25.jre7.jar’

postgresql-42.2.25. 100%[===================>] 961.78K  --.-KB/s    in 0.1s    

2022-04-10 22:21:12 (6.42 MB/s) - ‘postgresql-42.2.25.jre7.jar’ saved [984862/984862]



In [4]:
spark = SparkSession.builder.appName('Amazon Reviews ETL').config('spark.driver.extraClassPath', '/content/postgresql-42.2.25.jre7.jar').getOrCreate()

In [6]:
# user_df.show(20, truncate=False)
df = user_df

In [28]:
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [7]:
from pyspark.sql.functions import count

In [8]:
customers_table_df = df.groupBy('customer_id').agg(count('customer_id').alias('customer_count'))

In [45]:
customers_table_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [23]:
products_table_df = df.select(['product_id', 'product_title']).drop_duplicates(['product_id'])

In [47]:
products_table_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1060104342|Premium Asus MeMO...|
|159775045X|National Geograph...|
|5891090414|           STAND-N02|
|6175001532|Colorful Abstract...|
|9572548964|Pink Flower Purpl...|
+----------+--------------------+
only showing top 5 rows



In [48]:
# confirm duplicates are dropped
products_table_df.groupBy('product_id').agg(count('product_id').alias('id_count')).show()

+----------+--------+
|product_id|id_count|
+----------+--------+
|B00BURIAOA|       1|
|B007SLDF7O|       1|
|B00NF2RUTO|       1|
|B00OHRYFGC|       1|
|B0027531JI|       1|
|B00NOLH4NI|       1|
|B00JUXPHWE|       1|
|B00V3RR93U|       1|
|B00FAC02MO|       1|
|B00DPMNNUO|       1|
|B00YVVE6AE|       1|
|B00SB76Z5S|       1|
|B00X50ICEI|       1|
|B00OS8BJ9G|       1|
|B0107QPOFC|       1|
|B00Z0ME68A|       1|
|B00R1QYAW0|       1|
|B00MGZ120Y|       1|
|B00MWRNMEK|       1|
|B00PX06PTM|       1|
+----------+--------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import col, to_date

In [11]:
df.select(to_date(col('review_date'), 'yyyy-mm-dd')).tail(20)

KeyboardInterrupt: ignored

In [13]:
review_id_table_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date(col('review_date'), 'yyyy-mm-dd').alias('review_date')])

In [14]:
review_id_table_df = review_id_table_df.drop_duplicates(['review_id'])

In [15]:
print("Distinct count: "+str(review_id_table_df.count()))

Distinct count: 9002021


In [16]:
vine_table_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

In [17]:
vine_table_df = vine_table_df.withColumn('star_rating', vine_table_df.star_rating.cast('int'))

In [18]:
vine_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [19]:
 from getpass import getpass
mode = "append"
password = getpass('Enter Password')
jdbc_url="jdbc:postgresql://dataviz.cfqtepfdzy8v.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter Password··········


In [25]:
review_id_table_df.write.jdbc(url=jdbc_url,  table='review_id_table', mode=mode, properties=config)

In [ ]:
customers_table_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [24]:
products_table_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [21]:
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [22]:
vine_table_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

